In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as w

In [2]:
df = pd.read_csv('Desktop/Dashboard voila/train.csv')

In [3]:
df.Embarked.fillna('N_Info', inplace=True)
df.Cabin.fillna('N_Info', inplace=True)
df.Age.fillna(np.trunc(df.Age.mean()), inplace=True)
df.Survived = df.Survived.apply(lambda x: 'Yes' if x==1 else 'No')

In [4]:
Sex = list(df.Sex.drop_duplicates())
Embarked = list(df.Embarked.drop_duplicates())
Embarked.append('All')
Embarked.sort()
Survived = list(df.Survived.drop_duplicates())

In [5]:
#Checkbox Sexo
check = [w.Checkbox(value=False, description=z) for z in Sex]

In [6]:
#Slider Idade
df.Age = np.trunc(df.Age)
df.Age.fillna(np.trunc(df.Age.mean()), inplace=True)
df.Age = df.Age.astype(int)

age_min = df.Age.min()
age_max = df.Age.max()
age_slider = range(age_min, age_max)

w_age = w.SelectionRangeSlider(
    options=age_slider,
    index=(0, len(age_slider)-1),
    description='Age',
    disabled=False
)

In [7]:
#Dropdown
w_embarked = w.Dropdown(
    options=Embarked,
    value='All',
    description='Embarked',
    disabled=False
)

In [26]:
def slider(limites,Male, Female, Embarked):
    
    if Embarked !='All':
        df_E = df[df.Embarked==Embarked].copy()
    else:
        df_E = df.copy()
    
    df_plot = df_E[['Survived','Sex','PassengerId']][(df_E.Age>=limites[0])&(df_E.Age<= limites[1])].copy()
    
    if Male==True and Female==False:
        df_plot = df_plot[df_plot.Sex=='male'].copy()
        
    if Female==True and Male==False:
        df_plot = df_plot[df_plot.Sex=='female'].copy()
        
    df_table = df[df.index.isin(df_plot.index)]
   
    df_plot = df_plot.groupby(by=['Survived','Sex']).count().reset_index()

    fig1= px.bar(df_plot, y='PassengerId',x='Sex',color='Survived', labels=dict(PassengerId='Qt'),title='Survived Passagers Titanic',barmode='group')

    fig2 = go.Figure(data=[go.Table(
        header=dict(values=list(df_table.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[df_table.PassengerId,df_table.Survived,df_table.Pclass,
                           df_table.Name,df_table.Sex,df_table.Age,df_table.SibSp,
                           df_table.Parch,df_table.Ticket,df_table.Fare,df_table.Cabin,df_table.Embarked],
                   fill_color='lavender',
                   align='left'))
            ])

    fig1.update_layout(
    autosize=False,
    width=800,
    height=600)
    
    fig2.update_layout(height=300, margin=dict(r=15, l=5, t=5, b=5))
    
    return fig1.show(),fig2.show()

# Dashboard Interativo com python

## Exemplo basico da criação de dashboard apartir de arquivo csv somente com uso do python


## https://www.linkedin.com/in/bruno-o-santos/

In [27]:
interactive_plot = w.interactive(slider, limites=w_age, Male=check[0].value, Female=check[1].value, 
                                 Embarked = w_embarked)

output = interactive_plot.children[-1]
output.layout.height = '1000px'
display(interactive_plot)


interactive(children=(SelectionRangeSlider(description='Age', index=(0, 79), options=(0, 1, 2, 3, 4, 5, 6, 7, …